In [1]:
import wikipedia
import string
import re
import codecs
import collections
import sys
from itertools import islice, tee

### 0. Языки
Языки, для которых реализовано распознавание - горномарийский, якутский, татарский, чувашский:

In [2]:
langs = ('mrj', 'sah', 'tt', 'cv')

### 1. Обучение
Функция для скачивания статей из википедии (взято [отсюда](https://github.com/ElizavetaKuzmenko/Programming-and-computer-instruments/blob/master/nlp3year/LangDetect_3year.ipynb)):

In [3]:
def get_texts_for_lang(lang, n=10):
    wikipedia.set_lang(lang)
    wiki_content = []
    pages = wikipedia.random(n)
    for page_name in pages:
        try:
            page = wikipedia.page(page_name)
        except wikipedia.exceptions.WikipediaException:
            print('Skipping page {}'.format(page_name))
            continue

        wiki_content.append('{}\n{}'.format(page.title, page.content.replace('==', '')))

    return wiki_content

Скачиваем обучающие статьи для каждого языка (взято [отсюда](https://github.com/ElizavetaKuzmenko/Programming-and-computer-instruments/blob/master/nlp3year/LangDetect_3year.ipynb)):

In [4]:
wiki_texts = {}
for lang in langs:
    wiki_texts[lang] = get_texts_for_lang(lang, 100)
    print(lang, len(wiki_texts[lang]))

mrj 100
Skipping page Бэстээх (Хаҥалас улууһа)
sah 99


C:\Users\Андрей\Anaconda3\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 184 of the file C:\Users\Андрей\Anaconda3\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


Skipping page 79 (мәгънәләр)
Skipping page 964 (мәгънәләр)
Skipping page 104 (мәгънәләр)
Skipping page 279 (мәгънәләр)
tt 96
Skipping page Çăхан (пĕлтерĕшĕсем)
cv 99


Функция для токенизации (специфично для текстов на кириллице - удаляется вся стандартная латиница):

In [5]:
def tokenize(text):
    return filter(None,re.split('[ —–'+string.punctuation+'a-zA-Z0-9]+',text))

Функция для удаления повторов из словарей (in-place):

In [6]:
def symmetric_difference_dicts(d,d_keys):
    not_repeated = set(d[d_keys[0]].keys())
    for k in d_keys[1:]:
        not_repeated = not_repeated ^ set(d[k].keys())
    for k in d_keys:
        d[k] = {x:d[k][x] for x in not_repeated if x in d[k]}

Функция, возвращающая списки самых частотных элементов для языков:

In [7]:
#most frequent words in each language
def get_most_freq(d,d_keys,num):
    mostfreq = {}
    for k in d_keys:
        mostfreq[k] = set(sorted(d[k], key=lambda w: d[k][w], reverse=True)[:num])
    return mostfreq

Функция для получения списков самых частотных слов:

In [8]:
def most_frequent_words(texts,langs):
    freqs = {}
    for lang in wiki_texts:
        corpus = wiki_texts[lang]
        freqs[lang] = collections.defaultdict(lambda: 0)
        for article in corpus:
            for word in tokenize(article.replace('\n', '').lower()):
                freqs[lang][word] += 1
    
    symmetric_difference_dicts(freqs,langs)
   
    mostfreq = get_most_freq(freqs,langs,100)

    return mostfreq

Функция для определения языка текста словарным методом:

In [9]:
def dict_method(text,mostfreq):
    sums = {}
    for lang in mostfreq:
        sums[lang] = len([word for word in tokenize(text.replace('\n', '').lower()) if word in mostfreq[lang]])
    return sorted(sums, key=lambda w: sums[w], reverse=True)[0]

Составление n-грамм (взято [отсюда](https://github.com/ElizavetaKuzmenko/Programming-and-computer-instruments/blob/master/nlp3year/LangDetect_3year.ipynb)):

In [10]:
def make_ngrams(text):
    N = 3 # задаем длину n-граммы
    ngrams = zip(*(islice(seq, index, None) for index, seq in enumerate(tee(text, N))))
    ngrams = [''.join(x) for x in ngrams]
    return ngrams

Функция для получения списков самых частотных n-грамм:

In [11]:
def most_frequent_ngrams(texts,langs):
    freqs = {}
    for lang in wiki_texts:
        corpus = wiki_texts[lang]
        freqs[lang] = collections.defaultdict(lambda: 0)
        for article in corpus:
            for ngram in make_ngrams(article.replace('\n', '').lower()):
                freqs[lang][ngram] += 1
    
    symmetric_difference_dicts(freqs,langs)
   
    mostfreq = get_most_freq(freqs,langs,300)

    return mostfreq

Функция для определения языка текста методом n-грамм:

In [12]:
def ngram_method(text,mostfreq):
    sums = {}
    for lang in mostfreq:
        sums[lang] = len([ngram for ngram in make_ngrams(text.replace('\n', '').lower()) if ngram in mostfreq[lang]])
    return sorted(sums, key=lambda w: sums[w], reverse=True)[0]

Находим частотные слова и n-граммы:

In [13]:
mostfreqwords = most_frequent_words(wiki_texts,langs)
mostfreqngrams = most_frequent_ngrams(wiki_texts,langs)

### 2. Предсказание
Функция, открывающая файл и предсказывающая для него язык:

In [14]:
def predict_lang(filename,mfw,mfn):
    text = open(filename,'r',encoding='utf-8-sig').read()
    print('dict method:',dict_method(text,mfw))
    print('ngram method:',ngram_method(text,mfn))

Предскажем язык текста из файла text.txt:

In [16]:
predict_lang('text.txt',mostfreqwords,mostfreqngrams)

dict method: mrj
ngram method: mrj


Язык определен верно.
### 3. Отчет
Лучше работает метод n-грамм. Тест:

In [22]:
# test texts
test_texts = {}
for lang in langs:
    test_texts[lang] = get_texts_for_lang(lang, 100)
    print(lang, len(test_texts[lang]))

mrj 100
sah 100


C:\Users\Андрей\Anaconda3\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 184 of the file C:\Users\Андрей\Anaconda3\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


Skipping page TR
Skipping page 298 (мәгънәләр)
Skipping page Аксумла
tt 97
cv 100


In [23]:
#test
def test(textdict,mfw,mfn):
    errors_dict = 0
    errors_ngrams = 0
    texts = 0
    for lang in textdict:
        corpus = textdict[lang]
        for article in corpus:
            texts += 1
            if lang != dict_method(article,mfw):
                errors_dict += 1
            if lang != ngram_method(article,mfn):
                errors_ngrams += 1
    print('dict error rate:',errors_dict/texts)
    print('ngram error rate:',errors_ngrams/texts)

In [24]:
test(test_texts,mostfreqwords,mostfreqngrams)

dict error rate: 0.022670025188916875
ngram error rate: 0.0025188916876574307
